In [7]:
%pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 4.3 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from geopy.distance import geodesic

/var/folders/cv/bbbq9ykj0jg6pt6c7tx5_vjh0000gn/T/ipykernel_84602/25577971.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/cv/bbbq9ykj0jg6pt6c7tx5_vjh0000gn/T/ipykernel_84602/25577971.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [15]:
# intervention is parquet file
intervention = pd.read_parquet("/Users/camillecu/Downloads/KUL/MDA/project/preprocess_intervention/intvn_clean.parquet")

# input a geojson file
aed = gpd.read_file("/Users/camillecu/Downloads/KUL/MDA/project/Visualization/Created GeoJSON Files/AED_GEO.geojson")
aed = aed.to_crs(epsg=4326)

folder_path = "/Users/camillecu/Downloads/KUL/MDA/project/clustering/vector_shape"
# import amb, pit, mug 
amb = gpd.read_file(folder_path + "/amb.shp")
pit = gpd.read_file(folder_path + "/pit.shp")
mug = gpd.read_file(folder_path + "/mug.shp")

In [16]:
# concat amb, pit, mug to one dataframe
vecotrs = pd.concat([amb, pit, mug], ignore_index=True)
# convert geometry to lat and lng
# Remove null values
vecotrs = vecotrs.dropna(subset=['geometry'])

# Convert geometry to lat and lng
vecotrs['lat'] = vecotrs['geometry'].apply(lambda x: x.y)
vecotrs['lng'] = vecotrs['geometry'].apply(lambda x: x.x)

# only keep lat, lng and geometry
vecotrs = vecotrs[['lat', 'lng', 'geometry']]


In [17]:
aed = aed[['lat', 'lng']]
cardiac_location = intervention[intervention['eventtype firstcall'].isin(['P039 - Cardiac problem (other than thoracic pain)', 'P003 - Cardiac arrest'])]
cardiac_location = cardiac_location[['latitude intervention', 'longitude intervention', 'latitude permanence', 'longitude permanence']]


In [22]:
# Create a GeoDataFrame from the cardiac_location DataFrame
cardiac_location = gpd.GeoDataFrame(cardiac_location, geometry=gpd.points_from_xy(cardiac_location['latitude intervention'], cardiac_location['longitude intervention']))
cardiac_location.set_crs(epsg=4326, inplace=True)
# raname latitude intervention and longitude intervention to lat and lng
cardiac_location.rename(columns={'latitude intervention': 'lat', 'longitude intervention': 'lng'}, inplace=True)


In [23]:
cardiac_location

,lat,lng,latitude permanence,longitude permanence,geometry
16,51.29850,4.47723,51.29857,4.48800,POINT (51.29850 4.47723)
23,51.20685,5.13662,51.16013,4.99975,POINT (51.20685 5.13662)
49,51.24848,4.43869,51.22249,4.43629,POINT (51.24848 4.43869)
69,51.20303,4.34549,51.17118,4.31285,POINT (51.20303 4.34549)
98,51.05218,4.63286,51.13121,4.56334,POINT (51.05218 4.63286)
...,...,...,...,...,...
338419,50.45987,4.83580,50.46686,4.88765,POINT (50.45987 4.83580)
338456,50.33426,4.90938,50.32111,4.88316,POINT (50.33426 4.90938)
338460,50.51792,4.77706,50.46683,4.88732,POINT (50.51792 4.77706)
338467,50.71017,4.51832,50.66676,4.56167,POINT (50.71017 4.51832)


In [20]:
vecotrs

,lat,lng,geometry
0,51.215516,4.443788,POINT (4.44379 51.21552)
1,51.178831,4.361948,POINT (4.36195 51.17883)
2,51.232810,4.377237,POINT (4.37724 51.23281)
3,51.315516,4.321506,POINT (4.32151 51.31552)
4,51.213623,4.405718,POINT (4.40572 51.21362)
...,...,...,...
392,50.715566,6.007128,POINT (6.00713 50.71557)
393,50.159132,5.683118,POINT (5.68312 50.15913)
394,49.678012,5.821161,POINT (5.82116 49.67801)
395,49.923825,5.380069,POINT (5.38007 49.92383)


In [24]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Your Mapbox access token
mapbox_access_token = 'pk.eyJ1Ijoid2VuaGFuY3UiLCJhIjoiY2x3cTlrYjlnMDAybTJqczdzOWFwcWpjdyJ9.yNEGbGM7vNhqbdfdfFNTng'


# Create initial figure
fig = px.scatter_mapbox(cardiac_location, lat='lat', lon='lng',
                        center=dict(lat=50.8503, lon=4.3517),
                        zoom=7, mapbox_style="open-street-map")

fig.update_layout(mapbox=dict(accesstoken=mapbox_access_token))

# Define the layout
app.layout = html.Div([
    dcc.Graph(id='map', figure=fig, config={'scrollZoom': True}),
    dcc.Store(id='pin-location')
])

# Helper function to find the nearest point
def find_nearest_point(lat, lng, df):
    distances = df.apply(lambda row: geodesic((lat, lng), (row['lat'], row['lng'])).meters, axis=1)
    nearest_index = distances.idxmin()
    return df.loc[nearest_index, 'lat'], df.loc[nearest_index, 'lng']

# Callback to handle pin drop
@app.callback(
    Output('map', 'figure'),
    [Input('map', 'clickData')],
    prevent_initial_call=True
)
def update_map(clickData):
    if clickData is None:
        return dash.no_update

    lat = clickData['points'][0]['lat']
    lng = clickData['points'][0]['lon']

    # Find nearest AED
    nearest_aed_lat, nearest_aed_lng = find_nearest_point(lat, lng, aed)

    # Find nearest ambulance
    nearest_ambulance_lat, nearest_ambulance_lng = find_nearest_point(lat, lng, vecotrs)

    # Create the figure
    fig = px.scatter_mapbox(cardiac_location, lat='lat', lon='lng',
                            center=dict(lat=50.8503, lon=4.3517),
                            zoom=7, mapbox_style="open-street-map")
    fig.update_layout(mapbox=dict(accesstoken=mapbox_access_token))

    # Add the pin location
    fig.add_trace(go.Scattermapbox(
        lat=[lat],
        lon=[lng],
        mode='markers',
        marker=go.scattermapbox.Marker(size=14, color='red'),
        name='Dropped Pin'
    ))

    # Add the nearest AED location
    fig.add_trace(go.Scattermapbox(
        lat=[nearest_aed_lat],
        lon=[nearest_aed_lng],
        mode='markers',
        marker=go.scattermapbox.Marker(size=14, color='green'),
        name='Nearest AED'
    ))

    # Add the nearest ambulance location
    fig.add_trace(go.Scattermapbox(
        lat=[nearest_ambulance_lat],
        lon=[nearest_ambulance_lng],
        mode='markers',
        marker=go.scattermapbox.Marker(size=14, color='blue'),
        name='Nearest Ambulance'
    ))

    # Add lines connecting the pin to the nearest AED and ambulance
    fig.add_trace(go.Scattermapbox(
        lat=[lat, nearest_aed_lat],
        lon=[lng, nearest_aed_lng],
        mode='lines',
        line=dict(width=2, color='green'),
        name='Line to Nearest AED'
    ))

    fig.add_trace(go.Scattermapbox(
        lat=[lat, nearest_ambulance_lat],
        lon=[lng, nearest_ambulance_lng],
        mode='lines',
        line=dict(width=2, color='blue'),
        name='Line to Nearest Ambulance'
    ))

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

